In [35]:
import os
import json
import re
import shutil
input_folder = "encoded"
output_folder = "done_KNN"
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [29]:
# Use list comprehension to create a list of file paths
verilog_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith('.txt')]

# Print the list of file paths
print(len(verilog_files))
print(type(verilog_files[0]))
print(verilog_files[0])


289
<class 'str'>
encoded\adder11.txt


In [30]:
classes = ['adder', 'comparator', 'decoder', 'encoder','mult','mux','pe','sub','and','or','not','nand','nor','xnor']
label_mapping = {
        'adder': 0, 'comparator': 1, 'decoder': 2,
        'encoder': 3, 'mult': 4, 'mux': 5, 'pe': 6, 'sub': 7, 'and': 8, 'or': 9, 'not': 10, 'nand': 11, 'nor': 12, 'xnor': 13}

In [31]:
ground_truth_labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13] 
labels_torch = torch.tensor(ground_truth_labels)
one_hot_labels = F.one_hot(labels_torch, num_classes=14)


In [81]:
verilog_files_info = []
labels = []
labels_nums = []
for file_path in verilog_files:
    file_info = []
    input_count = 0
    output_count = 0
    reg_count = 0
    wire_count = 0
    subtr_count = 0
    shift_count = 0
    power_count = 0
    mux_count = 0
    mult_count = 0
    constVal_count = 0
    conditional_count = 0
    concat_count = 0
    case_count = 0
    not_count = 0
    nand_count = 0
    nor_count = 0
    xnor_count = 0
    xor_count = 0
    or_count = 0
    and_count = 0
    add_count = 0
    with open(file_path, "r") as file:
        file_name = file_path.split("\\")[-1].split(".")[0]
        matches = [string for string in label_mapping.keys() if re.match(f"^({string})\d+", file_name)]
        labels_nums.append(label_mapping[matches[0]])
        y = one_hot_labels[label_mapping[matches[0]]].tolist()
        labels.append(y)
        loaded_data = json.load(file)
        for sublist in loaded_data[0]:
            if sublist[0]: # input
                input_count +=1
                
            elif sublist[1]: #output
                output_count += 1
                
            elif sublist[2]: #reg
                reg_count +=1
                
            elif sublist[3]: #wire
                wire_count +=1
                
            elif sublist[4]:
                subtr_count += 1
                
            elif sublist[5]:
                shift_count +=1
                
            elif sublist[6]:
                mux_count+=1
                
            elif sublist[7]:
                mult_count+=1
                
            elif sublist[8]:
                constVal_count+=1
                
            elif sublist[9]:
                conditional_count+=1
                
            elif sublist[10]:
                concat_count +=1
                
            elif sublist[11]:
                case_count+=1
                
            elif sublist[12]:
                not_count+=1
                
            elif sublist[13]:
                nand_count+=1
                
            elif sublist[14]:
                nor_count+=1
                
            elif sublist[15]:
                xnor_count+=1
                
            elif sublist[16]:
                xor_count+=1
                
            elif sublist[17]:
                or_count+=1
                
            elif sublist[18]:
                and_count+=1
                
            elif sublist[19]:
                add_count+=1
                
            else: 
                print("eh el habal da")
        file_info.extend([input_count, output_count, reg_count, wire_count, subtr_count, shift_count, mux_count, mult_count, constVal_count, conditional_count, concat_count, case_count, not_count, nand_count, nor_count, xnor_count,xor_count, or_count, and_count, add_count])
    verilog_files_info.append(file_info)

verilog_files_info

[[3, 2, 0, 0, 0, 0, 16, 0, 64, 48, 0, 0, 0, 0, 0, 0, 0, 0, 32, 0],
 [2, 2, 0, 0, 0, 0, 0, 0, 8, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0],
 [3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 19, 33, 0],
 [9, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 19, 33, 0],
 [3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 4, 7, 0],
 [5, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 4, 7, 0],
 [3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 18, 0],
 [7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 18, 0],
 [2, 2, 0, 0, 0, 0, 0, 0, 8, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0],
 [3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 2, 0],
 [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
 [3, 2, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 0],
 [3, 2, 0, 0, 0, 0, 0, 0, 16, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0],
 [3, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 0],
 [2, 2, 0, 0, 0, 0, 8, 0, 24, 16, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0],
 [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [82]:
# len(verilog_files_info)
# len(labels)
verilog_files_info = np.array(verilog_files_info)
verilog_files_info.shape

(289, 20)

In [83]:
scaler = MinMaxScaler()

X = np.array(verilog_files_info)
X_normalized = scaler.fit_transform(X)
# print(type(X_normalized))
labels = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(X_normalized, labels, test_size=0.3, random_state=42)

# print(X_train)
# print(y_train)
knn = KNeighborsClassifier(n_neighbors=5)  # You can choose the number of neighbors
knn.fit(X_train, y_train)

knn_y_pred = knn.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_y_pred)
print("KNN Accuracy:", knn_accuracy)

KNN Accuracy: 0.20689655172413793


In [89]:
from sklearn import svm

scaler = MinMaxScaler()

X = np.array(verilog_files_info)
X_normalized = scaler.fit_transform(X)
labels_nums = np.array(labels_nums)

X_train, X_test, y_train, y_test = train_test_split(X_normalized, labels_nums, test_size=0.3, random_state=42)

svm_model = svm.SVC(kernel='linear')  # Using a linear kernel for simplicity

svm_model.fit(X_train, y_train)


y_pred = svm_model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.27586206896551724
